In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[11],9
commitnumber,1013f91,a22ab94
conc,-,0.0004
dv,-,0.001
klin,-,6.5e-24
molecule,{'co2': 0.0004},co2
ng_adju,-,"[0, 0]"
ng_refs,-,"[3, 3]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.135522                    1  0.284167
0.000750        2  0.011075                    2  0.093241
0.001052        3  0.008958                    3  0.050747
0.001476        4  0.003867                    4  0.010710
0.002070        5 -0.000992                    5 -0.016955
0.002904        6 -0.005214                    6 -0.034375
0.004074        7 -0.009058                    7 -0.039939
0.005714        8 -0.012832                    8 -0.041497
0.008015        9 -0.016731                    9 -0.042315
0.011243       10 -0.020919                   10 -0.042177
0.015771       11 -0.025537                   11 -0.043782
0.022122       12 -0.030826                   12 -0.045846
0.031031       13 -0.037804                   13 -0.051303
0.043528       14 -0.050266                   14 -0.062990
0.061057       15 -0.058666                   15 -0.068597
0.085645       16 -0.056174                   16 -0.059005
0.120136       17 -0.049593                   17 -0.041005
0.168516       18 -0.040878                   18 -0.021387
0.236378       19 -0.025044                   19 -0.002359
0.331549       20  0.003612                   20  0.016550
0.465100       21  0.061662                   21  0.052876
0.652400       22  0.178621                   22  0.150582
0.915100       23  0.208920                   23  0.174711
1.283650       24  0.148116                   24  0.137243
1.800600       25  0.102421                   25  0.116991
2.525700       26  0.060482                   26  0.085583
3.542800       27  0.035792                   27  0.056620
4.969550       28  0.021337                   28  0.028836
6.970850       29  0.011901                   29  0.009320
9.778100       30  0.005698                   30  0.003330
13.715850      31  0.001829                   31  0.004506
19.239350      32 -0.000290                   32  0.003434
26.987250      33 -0.001131                   33  0.000828
37.855300      34 -0.001156                   34 -0.000876
53.100050      35 -0.001794                   35 -0.001972
73.887500      36 -0.002404                   36 -0.002457
97.662500      37 -0.002777                   37 -0.002799
121.437500     38 -0.003084                   38 -0.003085
145.212500     39 -0.003142                   39 -0.003101
168.987500     40 -0.003400                   40 -0.003495
192.762500     41 -0.003643                   41 -0.003839
216.537500     42 -0.003854                   42 -0.004172
240.312500     43 -0.004027                   43 -0.004423
264.087500     44 -0.004161                   44 -0.004632
287.862500     45 -0.004257                   45 -0.004769
311.637500     46 -0.004313                   46 -0.004800
335.412500     47 -0.004332                   47 -0.004840
359.187500     48 -0.004311                   48 -0.004715
382.962500     49 -0.004252                   49 -0.004583
406.737500     50 -0.004153                   50 -0.004367
430.512500     51 -0.004013                   51 -0.004060
454.287500     52 -0.003833                   52 -0.003818
478.062500     53 -0.003612                   53 -0.003310
501.837500     54 -0.003347                   54 -0.003018
525.612500     55 -0.003035                   55 -0.002359
549.387500     56 -0.002672                   56 -0.001934
573.162500     57 -0.002251                   57 -0.001204
596.937500     58 -0.001750                   58 -0.000535
620.712500     59 -0.001134                   59  0.000277
644.487500     60 -0.000318                   60  0.001264
668.262500     61  0.000778                   61  0.002462
692.037500     62  0.002005                   62  0.003712
715.812500     63  0.002763                   63  0.004189
739.587500     64  0.002633                   64  0.003964
763.362500     65  0.001974                   65  0.002942
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -6.709430  0.000000 -6.709430 -6.609555  7.849093e-09   
0.000624    2     -6.709434  0.000014 -6.709420 -6.609563  2.821062e-05   
0.000876    3     -6.709436  0.000017 -6.709419 -6.609566  3.422732e-05   
0.001229    4     -6.709439  0.000020 -6.709419 -6.609570  4.056964e-05   
0.001723    5     -6.709443  0.000024 -6.709419 -6.609576  4.677310e-05   
0.002417    6     -6.709448  0.000029 -6.709419 -6.609583  5.270772e-05   
0.003391    7     -6.709454  0.000034 -6.709420 -6.609593  5.860621e-05   
0.004757    8     -6.709462  0.000041 -6.709421 -6.609606  6.537010e-05   
0.006672    9     -6.709474  0.000050 -6.709424 -6.609624  7.383710e-05   
0.009359    10    -6.709490  0.000061 -6.709429 -6.609649  8.494815e-05   
0.013128    11    -6.709513  0.000074 -6.709439 -6.609682  1.000714e-04   
0.018415    12    -6.709544  0.000090 -6.709455 -6.609730  1.206385e-04   
0.025830    13    -6.709590  0.000108 -6.709482 -6.609799  1.489695e-04   
0.036232    14    -6.709658  0.000130 -6.709528 -6.609901  1.872787e-04   
0.050823    15    -6.709772  0.000157 -6.709615 -6.610062  2.401910e-04   
0.071291    16    -6.709966  0.000209 -6.709757 -6.610321  3.322960e-04   
0.100000    17    -6.710276  0.000328 -6.709948 -6.610709  5.201302e-04   
0.140271    18    -6.710752  0.000567 -6.710185 -6.611271  8.861943e-04   
0.196760    19    -6.711466  0.001008 -6.710459 -6.612073  1.544986e-03   
0.275997    20    -6.712509  0.001815 -6.710694 -6.613199  2.649052e-03   
0.387100    21    -6.713938  0.003292 -6.710646 -6.614718  4.385875e-03   
0.543100    22    -6.715495  0.005988 -6.709507 -6.616407  7.051793e-03   
0.761700    23    -6.715462  0.010580 -6.704881 -6.616417  1.096186e-02   
1.068500    24    -6.712944  0.015655 -6.697289 -6.613924  1.481837e-02   
1.498800    25    -6.709108  0.019369 -6.689739 -6.610189  1.808004e-02   
2.102400    26    -6.704374  0.021959 -6.682416 -6.605561  2.181758e-02   
2.949000    27    -6.699595  0.023245 -6.676350 -6.600742  2.558273e-02   
4.136600    28    -6.695038  0.023723 -6.671315 -6.595874  2.868035e-02   
5.802500    29    -6.690734  0.023630 -6.667104 -6.591009  2.950615e-02   
8.139200    30    -6.686743  0.022933 -6.663810 -6.586159  2.723683e-02   
11.417000   31    -6.683157  0.021559 -6.661598 -6.581520  2.389085e-02   
16.014700   32    -6.680098  0.019496 -6.660602 -6.577299  2.212435e-02   
22.464000   33    -6.677717  0.016893 -6.660824 -6.573703  2.115210e-02   
31.510500   34    -6.676143  0.014108 -6.662035 -6.570762  1.909782e-02   
44.200100   35    -6.675255  0.011482 -6.663773 -6.568151  1.516908e-02   
62.000000   36    -6.676157  0.008602 -6.667555 -6.567179  1.003875e-02   
85.775000   37    -6.680320  0.005994 -6.674326 -6.570195  6.135195e-03   
109.550000  38    -6.686934  0.004788 -6.682146 -6.576721  4.777777e-03   
133.325000  39    -6.696104  0.005272 -6.690832 -6.586561  5.928906e-03   
157.100000  40    -6.707379  0.007698 -6.699680 -6.598731  9.364331e-03   
180.875000  41    -6.720850  0.011593 -6.709256 -6.613291  1.407937e-02   
204.650000  42    -6.736595  0.017079 -6.719516 -6.630276  2.025246e-02   
228.425000  43    -6.754673  0.024302 -6.730371 -6.649818  2.804248e-02   
252.200000  44    -6.775111  0.033399 -6.741712 -6.671887  3.765426e-02   
275.975000  45    -6.797929  0.044498 -6.753431 -6.696527  4.924869e-02   
299.750000  46    -6.823134  0.057715 -6.765419 -6.723751  6.304116e-02   
323.525000  47    -6.850720  0.073153 -6.777567 -6.753437  7.920875e-02   
347.300000  48    -6.880669  0.090902 -6.789767 -6.785740  9.787819e-02   
371.075000  49    -6.912954  0.111045 -6.801909 -6.820499  1.193553e-01   
394.850000  50    -6.947538  0.133652 -6.813885 -6.857705  1.436535e-01   
418.625000  51    -6.984367  0.158786 -6.825581 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')